In [11]:
#this program will apply the bias subtraction,
#flat fielding and subtract the gradient background,
#it will also align the images to the first image in the list

#if you use this code, please cite Oelkers et al. 2015, AJ, 149, 50

#import the relevant libraries for basic tools
import numpy as np
import scipy
from scipy import stats
import scipy.ndimage as ndimage
import astropy
from astropy.io import fits
from astropy.nddata.utils import Cutout2D
from astropy.wcs import WCS
import math
import time
from photutils import centroid_com, centroid_1dg, centroid_2dg

#libraries for image registration
import FITS_tools
from FITS_tools.hcongrid import hcongrid

#import relevant libraries for a list
import glob, os
from os import listdir
from os.path import isfile, join

#import relevant spline libraries
from scipy.interpolate import Rbf

In [2]:
user = input('Insert the name of your user')
print('the user is',user)

Insert the name of your userusuario
the user is usuario


In [12]:
#####UPDATE INFORMATION HERE####
#DO YOU WANT TO FLAT FIELD AND BIAS SUBTRACT?
align = 1# yes = 1 no = 0 to align based on coordinates

#useful directories
rawdir = '/home/'+user+'/Documents/Mestrado/FFIs/8_golden/teste/' #directory with the raw images
cdedir = '/home/'+user+'/dia_kepler/mychanges/Python/' #directory where the code 'lives'
clndir = '/home/'+user+'/Documents/Mestrado/FFIs/8_golden/teste/teste_clean/'#directory for the cleaned images to be output

#sample every how many pixels? usually 32x32 is OK but it can be larger or smaller
pix = 25 # UPDATE HERE FOR BACKGROUND SPACING
axs1 = 1100 # UPDATE HERE FOR IMAGE AXIS SIZE - number of columns
axs2 = 1024 #number of lines
###END UPDATE INFORMATION###

#get the image list and the number of files which need reduction
os.chdir(rawdir) #changes to the raw image direcotory
files = [f for f in glob.glob("*.fits") if isfile(join(rawdir, f))] #gets the relevant files with the proper extension
files.sort()
nfiles = len(files)
os.chdir(cdedir) #changes back to the code directory

#get the zeroth image for registration
#read in the image
ref, rhead = fits.getdata(rawdir+files[0], header = True)
#rhead['CRPIX1'] = 1001.
#rhead['NAXIS1'] = 1100
#rhead['NAXIS2'] = 1024

#sample every how many pixels?
bxs = 275 #how big do you want to make the boxes for each image? # 1100/4
lop = 2*pix
sze = (bxs/pix)*(bxs/pix)+2*(bxs/pix)+1 #size holder for later
files

['kplr2009114174833_ffi-cal.fits',
 'kplr2009114204835_ffi-cal.fits',
 'kplr2009115002613_ffi-cal.fits',
 'kplr2009115053616_ffi-cal.fits',
 'kplr2009115080620_ffi-cal.fits',
 'kplr2009115131122_ffi-cal.fits',
 'kplr2009115173611_ffi-cal.fits',
 'kplr2009116035924_ffi-cal.fits']

In [ ]:
#begin cleaning
for ii in range(0, nfiles):
    hld = files[ii].split('.') #gives an array like ['kplr2009114174833_ffi-cal', 'fits'

    #update the name to be appropriate for what was done to the file
    if (align == 1):
        finnme = hld[0]+'_sa.fits'
    if os.path.isfile(clndir+finnme) == 0:#if there is not an image in the cleaning directory
        st = time.time()
        sts = time.strftime("%c")
        print ('Now cleaning '+files[ii]+' at '+sts+'.')
        #read in the image
        hdulist = fits.open(rawdir+files[ii])
        for j in range(1,len(hdulist)):#counting on that there is an primary header (the zero in the list)
            orgimg, header = fits.getdata(rawdir+files[ii], header = True)
            print(np.shape(orgimg))
            w = WCS(header)


            cut = Cutout2D(orgimg, (550,512), (axs2, axs1), wcs = w) #1 arg is the center collumn
            bigimg = cut.data
            #get the holders ready
            res = np.zeros(shape=(axs2, axs1)) #holder for the background 'image'
            bck = np.zeros(shape=int((axs1/bxs)**2)) #get the holder for the image background
            sbk = np.zeros(shape=int((axs1/bxs)**2)) #get the holder for the sigma of the image background

            tts = 0
            for oo in range(0, axs2, bxs):#axs1=1100 axs2=1024
                for ee in range(0, axs1, bxs):
                    img = bigimg[ee:ee+bxs, oo:oo+bxs] #split the image into small subsections

                    #calculate the sky statistics
                    #from astropy.stats import sigma_clipped_stats
                    # mean, median, std = sigma_clipped_stats(data, sigma=3.0)
                    cimg, clow, chigh = scipy.stats.sigmaclip(img, low=2.5, high = 2.5) #do a 2.5 sigma clipping
                    sky = np.median(cimg) #determine the sky value
                    sig = np.std(cimg) #determine the sigma(sky)

                    bck[tts] = sky #insert the image median background
                    sbk[tts] = sig #insert the image sigma background

                    #create holder arrays for good and bad pixels
                    x = np.zeros(shape=int(sze))
                    y = np.zeros(shape=int(sze))
                    v = np.zeros(shape=int(sze))
                    s = np.zeros(shape=int(sze))
                    nd = 0

#                     #begin the sampling of the "local" sky value
#                     for jj in range(0, bxs+pix, pix):
#                         for kk in range(0,bxs+pix, pix):
#                             il = np.amax([jj-lop,0])
#                             ih = np.amin([jj+lop, bxs-1])
#                             jl = np.amax([kk-lop, 0])
#                             jh = np.amin([kk+lop, bxs-1])
#                             c = img[jl:jh, il:ih]
#                             #select the median value with clipping
#                             cc, cclow, cchigh = scipy.stats.sigmaclip(c, low=2.5, high = 2.5) #sigma clip the background
#                             lsky = np.median(cc) #the sky background
#                             ssky = np.std(cc) #sigma of the sky background
#                             x[nd] = np.amin([jj, bxs-1]) #determine the pixel to input
#                             y[nd] = np.amin([kk, bxs-1]) #determine the pixel to input
#                             v[nd] = lsky #median sky
#                             s[nd] = ssky #sigma sky
#                             nd = nd + 1

#                     #now we want to remove any possible values which have bad sky values
#                     rj = np.where(v <= 0) #stuff to remove
#                     kp = np.where(v > 0) #stuff to keep

#                     if (len(rj[0]) > 0):
#                         #keep only the good points
#                         xgood = x[kp]
#                         ygood = y[kp]
#                         vgood = v[kp]
#                         sgood = s[kp]

#                         for jj in range(0, len(rj[0])):
#                             #select the bad point
#                             xbad = x[rj[jj]]
#                             ybad = y[rj[jj]]
#                             #use the distance formula to get the closest points
#                             rd = math.sqrt((xgood-ygood)**2.+(ygood-ybad)**2.)
#                             #sort the radii
#                             pp = sorted(range(len(rd)), key = lambda k:rd[k])
#                             #use the closest 10 points to get a median
#                             vnear = vgood[pp[0:9]]
#                             ave = np.median(vnear)
#                             #insert the good value into the array
#                             v[rj[jj]] = ave

#                     #now we want to remove any possible values which have bad sigmas
#                     rjs = np.where(s >= 2*sig)
#                     rj  = rjs[0]
#                     kps = np.where(s < 2*sig)
#                     kp  = kps[0]

#                     if (len(rj) > 0):
#                         #keep only the good points
#                         xgood = np.array(x[kp])
#                         ygood = np.array(y[kp])
#                         vgood = np.array(v[kp])
#                         sgood = np.array(s[kp])

#                         for jj in range(0, len(rj)):
#                             #select the bad point
#                             xbad = x[rj[jj]]
#                             ybad = y[rj[jj]]
#                             #print xbad, ybad
#                             #use the distance formula to get the closest points
#                             rd = np.sqrt((xgood-xbad)**2.+(ygood-ybad)**2.)
#                             #sort the radii
#                             pp = sorted(range(len(rd)), key = lambda k:rd[k])
#                             #use the closest 10 points to get a median
#                             vnear = vgood[pp[0:9]]
#                             ave = np.median(vnear)
#                             #insert the good value into the array
#                             v[rj[jj]] = ave

#                     #now we interpolate to the rest of the image with a thin-plate spline    
#                     xi = np.linspace(0, bxs-1, bxs)
#                     yi = np.linspace(0, bxs-1, bxs)
#                     XI, YI = np.meshgrid(xi, yi)
#                     rbf = Rbf(x, y, v, function = 'thin-plate', smooth = 0.0)
#                     reshld = rbf(XI, YI)

#                     #now add the values to the residual image
#                     res[ee:ee+bxs, oo:oo+bxs] = reshld
#                     tts = tts+1

#             #get the median background
#             mbck = np.median(bck)
#             sbck = np.median(sbk)

#             #subtract the sky gradient and add back the median background
#             sub = bigimg-res
#             sub = sub + mbck

#             #align the image
#             algn = hcongrid(sub, header, rhead)

#             #update the header
#             header['CTYPE1'] = rhead['CTYPE1']
#             header['CTYPE2'] = rhead['CTYPE2']
#             header['CRVAL1'] = rhead['CRVAL1']
#             header['CRVAL2'] = rhead['CRVAL2']
#             header['CRPIX1'] = rhead['CRPIX1']
#             header['CRPIX2'] = rhead['CRPIX2']
#             header['CD1_1'] = rhead['CD1_1']
#             header['CD1_2'] = rhead['CD1_2']
#             header['CD2_1'] = rhead['CD2_1']
#             header['CD2_2'] = rhead['CD2_2']

#             #update the header
#             header['medback'] = mbck
#             header['sigback'] = sbck
#             header['bksub'] = 'yes'
#             if (align == 1):
#                 header['align'] = 'yes'

#             #write out the subtraction
#             shd = fits.PrimaryHDU(algn, header=header)
#             shd.writeto(clndir+finnme, overwrite = True)
#             #stop the watch
#             fn = time.time()
#             print ('Background subtraction for '+files[ii]+' finished in '+str(fn-st)+'s.')

# print ('All done! See ya later alligator.')

Now cleaning kplr2009114174833_ffi-cal.fits at Tue Feb 12 19:25:15 2019.
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(1024, 1100)
(102

In [9]:
from astropy.stats import sigma_clipped_stats
mean, median, std = sigma_clipped_stats(data, sigma=3.0)

NameError: name 'data' is not defined